In [1]:
import sys
sys.path.append('../')
from pathlib import Path
import pandas as pd
import numpy as np

from scripts.base import Feature, get_arguments, generate_features
Feature.dir = "F:/Kaggle/ventilator-pressure-prediction/data/input/features"

In [2]:
INPUT_DIR = Path("F:/Kaggle/ventilator-pressure-prediction/data/input/")
OUTPUT_DIR = Path('F:/Kaggle/ventilator-pressure-prediction/data/output/')
FEAT_DIR = Path("F:/Kaggle/ventilator-pressure-prediction/data/input/features/")

def get_train_test():
    _train = pd.read_csv(INPUT_DIR / "train.csv")
    _test = pd.read_csv(INPUT_DIR / "test.csv")
    return _train, _test

def load_datasets(feats):
    dfs = [pd.read_feather(FEAT_DIR / f'{f}_train.ftr') for f in feats]
    X_train = pd.concat(dfs, axis=1)
    dfs = [pd.read_feather(FEAT_DIR / f'{f}_test.ftr') for f in feats]
    X_test = pd.concat(dfs, axis=1)
    return X_train, X_test

In [ ]:
class Base(Feature):
    """結合用のデータ"""
    def create_features(self):
        self.train = train[['id', 'breath_id', 'time_step', 'u_in', 'u_out', 'pressure', 'fold']].copy()
        self.test = test[['id', 'breath_id', 'time_step', 'u_in', 'u_out']].copy()
        
train, test = get_train_test()
train_v2 = pd.read_csv(INPUT_DIR / "train_v2.csv")
train = pd.concat([train, train_v2[["fold"]]],axis=1)
#Base().run().save()

In [ ]:
class Area_cumsum(Feature):
    """u_inの下の面積っぽいけどなんか式違いそう"""
    def create_features(self):
        def create(df):
            df['area_cumsum'] = df['time_step'] * df['u_in']
            df['area_cumsum'] = df.groupby('breath_id')['area_cumsum'].cumsum()
            return df
        cols = ['area_cumsum']
        self.train = create(train)[cols]
        self.test = create(test)[cols]

train, test = get_train_test()
#Area_cumsum().run().save()

In [ ]:
class Area(Feature):
    """その時点までのu_in曲線の積分、変化量、時間の変化量"""
    def create_features(self):
        def create(df):
            df['time_delta'] = df.groupby('breath_id')['time_step'].diff().fillna(0)
            df['delta'] = df['time_delta'] * df['u_in']
            df['area'] = df.groupby('breath_id')['delta'].cumsum()
            df = df.fillna(0)
            return df
        cols = ['time_delta','delta','area']
        self.train = create(train)[cols]
        self.test = create(test)[cols]
        
train, test = get_train_test()
#Area().run().save()

In [ ]:
class Cross(Feature):
    """よくわからない"""
    def create_features(self):
        self.train['cross']  = train['u_in']*train['u_out'] 
        self.train['cross2'] = train['time_step']*train['u_out']
        self.test['cross']  = test['u_in']*test['u_out']
        self.test['cross2'] = test['time_step']*test['u_out']
        
train, test = get_train_test()
Cross().run().save()

In [ ]:
class U_in_cumsum_mean(Feature):
    """累積和と累積平均"""
    def create_features(self):
        def create(df):
            df['u_in_cumsum'] = (df['u_in']).groupby(df['breath_id']).cumsum()# 累積和
            df['one'] = 1
            df['count'] = (df['one']).groupby(df['breath_id']).cumsum()
            df['u_in_cummean'] =df['u_in_cumsum'] /df['count']
            return df
        self.train = create(train)[['u_in_cumsum', 'u_in_cummean']]
        self.test = create(test)[['u_in_cumsum', 'u_in_cummean']]
        
train, test = get_train_test()
U_in_cumsum_mean().run().save()

In [ ]:
class U_in_Lag(Feature):
    """累積和と累積平均"""
    def create_features(self):
        def create(df):
            df['u_in_lag'] = df.groupby('breath_id')['u_in'].shift(1)
            df['u_in_lag2'] = df.groupby('breath_id')['u_in'].shift(2)
            df['u_in_lag3'] = df.groupby('breath_id')['u_in'].shift(3)
            df['u_in_lag4'] = df.groupby('breath_id')['u_in'].shift(4)
            df['u_in_lag_back'] = df.groupby('breath_id')['u_in'].shift(-1)
            df['u_in_lag_back2'] = df.groupby('breath_id')['u_in'].shift(-2)
            df['u_in_lag_back3'] = df.groupby('breath_id')['u_in'].shift(-3)
            df['u_in_lag_back4'] = df.groupby('breath_id')['u_in'].shift(-4)
            df = df.fillna(0)
            return df
        cols = ['u_in_lag','u_in_lag2','u_in_lag3','u_in_lag4','u_in_lag_back','u_in_lag_back2','u_in_lag_back3','u_in_lag_back4']
        self.train = create(train)[cols]
        self.test = create(test)[cols]
        
train, test = get_train_test()
U_in_Lag().run().save()

In [ ]:
class U_out_Lag(Feature):
    """累積和と累積平均"""
    def create_features(self):
        def create(df):
            df['u_out_lag'] = df.groupby('breath_id')['u_out'].shift(1)
            df['u_out_lag2'] = df.groupby('breath_id')['u_out'].shift(2)
            df['u_out_lag3'] = df.groupby('breath_id')['u_out'].shift(3)
            df['u_out_lag4'] = df.groupby('breath_id')['u_out'].shift(4)
            df['u_out_lag_back'] = df.groupby('breath_id')['u_out'].shift(-1)
            df['u_out_lag_back2'] = df.groupby('breath_id')['u_out'].shift(-2)
            df['u_out_lag_back3'] = df.groupby('breath_id')['u_out'].shift(-3)
            df['u_out_lag_back4'] = df.groupby('breath_id')['u_out'].shift(-4)
            df = df.fillna(0)
            return df
        cols = ['u_out_lag','u_out_lag2','u_out_lag3','u_out_lag4','u_out_lag_back','u_out_lag_back2','u_out_lag_back3','u_out_lag_back4']
        self.train = create(train)[cols]
        self.test = create(test)[cols]
        
train, test = get_train_test()
U_out_Lag().run().save()

In [ ]:
class RC_OHE(Feature):
    """
    RとCのOHE
    """
    def create_features(self):
        def create(df):
            df['R'] = df['R'].astype(str)
            df['C'] = df['C'].astype(str)
            df['RC'] = df['R']+df['C']
            df = pd.get_dummies(df)
            return df
        cols = ['R_20', 'R_5', 'R_50', 'C_10', 'C_20', 'C_50', 'RC_2010', 'RC_2020', 'RC_2050', 'RC_5010', 'RC_5020', 'RC_5050', 'RC_510', 'RC_520', 'RC_550']
        self.train = create(train)[cols]
        self.test = create(test)[cols]
        
train, test = get_train_test()
RC_OHE().run().save()

In [ ]:
class U_out_stat(Feature):
    """
    u_outが0のときと1のときのu_inのmean,std,max
    breath_idごとに値はおなじになる
    """
    def create_features(self):
        def create(df):
            df = df.merge(
                df[df["u_out"]==0].groupby('breath_id')['u_in'].agg(["mean", "std", "max"]).add_prefix("u_out0_").reset_index(),
                on="breath_id"
                )
            df = df.merge(
                df[df["u_out"]==1].groupby('breath_id')['u_in'].agg(["mean", "std", "max"]).add_prefix("u_out1_").reset_index(),
                on="breath_id"
                )
            return df
        cols = ['u_out0_mean', 'u_out0_max', 'u_out0_std', 'u_out1_mean', 'u_out1_max', 'u_out1_std']
        self.train = create(train)[cols]
        self.test = create(test)[cols]

train, test = get_train_test()
U_out_stat().run().save()

In [ ]:
class U_inout_max(Feature):
    """
    u_out関係なくu_inのmaxとu_outのmax
    """
    def create_features(self):
        def create(df):
            df['breath_id__u_in__max'] = df.groupby(['breath_id'])['u_in'].transform('max')
            df['breath_id__u_out__max'] = df.groupby(['breath_id'])['u_out'].transform('max')
            return df
        cols = ['breath_id__u_in__max', 'breath_id__u_out__max']
        self.train = create(train)[cols]
        self.test = create(test)[cols]

train, test = get_train_test()
U_inout_max().run().save()

In [3]:
"""
https://www.kaggle.com/marutama/eda-about-pressure-with-colored-charts
u_inのstat
"""
class U_in_stat(Feature):
    """
    u_inのstat
    """
    def create_features(self):
        def create(df):
            u_in_max_dict = df.groupby('breath_id')['u_in'].max().to_dict()
            df['u_in_max'] = df['breath_id'].map(u_in_max_dict)
            u_in_min_dict = df.groupby('breath_id')['u_in'].min().to_dict()
            df['u_in_min'] = df['breath_id'].map(u_in_min_dict)
            u_in_mean_dict = df.groupby('breath_id')['u_in'].mean().to_dict()
            df['u_in_mean'] = df['breath_id'].map(u_in_mean_dict)
            u_in_std_dict = df.groupby('breath_id')['u_in'].std().to_dict()
            df['u_in_std'] = df['breath_id'].map(u_in_std_dict)
            return df
        cols = ['u_in_max', 'u_in_min','u_in_mean','u_in_std']
        self.train = create(train)[cols]
        self.test = create(test)[cols]

train, test = get_train_test()
U_in_stat().run().save()

[U_in_stat] start
[U_in_stat] done in 2 s


In [9]:
"""
https://www.kaggle.com/marutama/eda-about-pressure-with-colored-charts
u_inのstat
"""
class U_in_half_stat(Feature):
    """
    u_inのstat
    """
    def create_features(self):
        def create(df):
            # u_in_half is time:0 - time point of u_out:1 rise (almost 1.0s)
            df['tmp'] = df['u_out']*(-1)+1 # inversion of u_out
            df['u_in_half'] = df['tmp'] * df['u_in']

            # u_in_half: max, min, mean, std
            u_in_half_max_dict = df.groupby('breath_id')['u_in_half'].max().to_dict()
            df['u_in_half_max'] = df['breath_id'].map(u_in_half_max_dict)
            u_in_half_min_dict = df.groupby('breath_id')['u_in_half'].min().to_dict()
            df['u_in_half_min'] = df['breath_id'].map(u_in_half_min_dict)
            u_in_half_mean_dict = df.groupby('breath_id')['u_in_half'].mean().to_dict()
            df['u_in_half_mean'] = df['breath_id'].map(u_in_half_mean_dict)
            u_in_half_std_dict = df.groupby('breath_id')['u_in_half'].std().to_dict()
            df['u_in_half_std'] = df['breath_id'].map(u_in_half_std_dict)
            return df
        cols = ['u_in_half', 'u_in_half_max', 'u_in_half_min', 'u_in_half_mean', 'u_in_half_std']
        self.train = create(train)[cols]
        self.test = create(test)[cols]

train, test = get_train_test()
U_in_half_stat().run().save()

[U_in_half_stat] start
[U_in_half_stat] done in 2 s


In [18]:
"""
https://www.kaggle.com/marutama/eda-about-pressure-with-colored-charts
u_inのstat
"""
class U_in_first_last(Feature):
    """
    u_inのstat
    """
    def create_features(self):
        def create(df):
            # All entries are first point of each breath_id
            first_df = df.loc[0::80,:]
            # All entries are first point of each breath_id
            last_df = df.loc[79::80,:]

            # u_in: first point, last point
            u_in_first_dict = dict(zip(first_df['breath_id'], first_df['u_in']))
            df['u_in_first'] = df['breath_id'].map(u_in_first_dict)
            u_in_last_dict = dict(zip(first_df['breath_id'], last_df['u_in']))
            df['u_in_last'] = df['breath_id'].map(u_in_last_dict)
            # time(sec) of end point
            time_end_dict = dict(zip(last_df['breath_id'], last_df['time_step']))     
            df['time_end'] = df['breath_id'].map(time_end_dict)
            return df
        cols = ['u_in_first', 'u_in_last', 'time_end']
        self.train = create(train)[cols]
        self.test = create(test)[cols]

train, test = get_train_test()
U_in_first_last().run().save()

[U_in_first_last] start
[U_in_first_last] done in 1 s


In [20]:
feats = ['base', 'U_in_Lag']
train, test = load_datasets(feats)
train.head()

,id,breath_id,time_step,u_in,u_out,pressure,fold,u_in_first,u_in_last,time_end
0,1,1,0.000000,0.083334,0,5.837492,4,0.083334,4.987079,2.689766
1,2,1,0.033652,18.383041,0,5.907794,4,0.083334,4.987079,2.689766
2,3,1,0.067514,22.509278,0,7.876254,4,0.083334,4.987079,2.689766
3,4,1,0.101542,22.808822,0,11.742872,4,0.083334,4.987079,2.689766
4,5,1,0.135756,25.355850,0,12.234987,4,0.083334,4.987079,2.689766


In [ ]:
class U_in_Lag_Diff(Feature):
    """
    """
    def create_features(self):
        def create(df):
            df["u_in_lag1_diff"] = df["u_in"] - df["u_in_lag"]
            df["u_in_lag2_diff"] = df["u_in"] - df["u_in_lag2"]
            df["u_in_lag3_diff"] = df["u_in"] - df["u_in_lag3"]
            df["u_in_lag4_diff"] = df["u_in"] - df["u_in_lag4"]
            return df
        cols = ['u_in_lag1_diff', 'u_in_lag2_diff','u_in_lag3_diff', 'u_in_lag4_diff']
        self.train = create(train)[cols]
        self.test = create(test)[cols]
        display(self.train)

feats = ['Base', 'U_in_Lag']
train, test = load_datasets(feats)
display(train)
U_in_Lag_Diff().run().save()

In [ ]:
class U_in_Rolling(Feature):
    """
    """
    def create_features(self):
        def create(df):
            df["u_in_rolling_mean2"] = df[["breath_id", "u_in"]].groupby("breath_id").rolling(2).mean()["u_in"].reset_index(drop=True)
            df["u_in_rolling_mean4"] = df[["breath_id", "u_in"]].groupby("breath_id").rolling(4).mean()["u_in"].reset_index(drop=True)
            df["u_in_rolling_mean10"] = df[["breath_id", "u_in"]].groupby("breath_id").rolling(10).mean()["u_in"].reset_index(drop=True)
            
            df["u_in_rolling_max2"] = df[["breath_id", "u_in"]].groupby("breath_id").rolling(2).max()["u_in"].reset_index(drop=True)
            df["u_in_rolling_max4"] = df[["breath_id", "u_in"]].groupby("breath_id").rolling(4).max()["u_in"].reset_index(drop=True)
            df["u_in_rolling_max10"] = df[["breath_id", "u_in"]].groupby("breath_id").rolling(10).max()["u_in"].reset_index(drop=True)
            df["u_in_rolling_min2"] = df[["breath_id", "u_in"]].groupby("breath_id").rolling(2).min()["u_in"].reset_index(drop=True)
            df["u_in_rolling_min4"] = df[["breath_id", "u_in"]].groupby("breath_id").rolling(4).min()["u_in"].reset_index(drop=True)
            df["u_in_rolling_min10"] = df[["breath_id", "u_in"]].groupby("breath_id").rolling(10).min()["u_in"].reset_index(drop=True)
            df["u_in_rolling_std2"] = df[["breath_id", "u_in"]].groupby("breath_id").rolling(2).std()["u_in"].reset_index(drop=True)
            df["u_in_rolling_std4"] = df[["breath_id", "u_in"]].groupby("breath_id").rolling(4).std()["u_in"].reset_index(drop=True)
            df["u_in_rolling_std10"] = df[["breath_id", "u_in"]].groupby("breath_id").rolling(10).std()["u_in"].reset_index(drop=True)
            for col in df.columns:
                df[col] = df[col].fillna(df[col].mean())
            return df
        cols = ['u_in_rolling_mean2', 'u_in_rolling_mean4','u_in_rolling_mean10', 'u_in_rolling_max2', 'u_in_rolling_max4', 'u_in_rolling_max10',
                'u_in_rolling_min2', 'u_in_rolling_min4', 'u_in_rolling_min10', 'u_in_rolling_std2', 'u_in_rolling_std4', 'u_in_rolling_std10']
        self.train = create(train)[cols]
        self.test = create(test)[cols]
        display(self.train)

train, test = get_train_test()
U_in_Rolling().run().save()

In [ ]:
class U_in_delta(Feature):
# https://www.kaggle.com/c/ventilator-pressure-prediction/discussion/273974
    """
    """
    def create_features(self):
        def create(df):
            df['u_in_change']= df['u_in'].shift(-1, fill_value=0)-df['u_in']
            df['time_delta'] = df.groupby('breath_id')['time_step'].diff().fillna(0)
            df['area_u_in_abs']=df['u_in_change']*df['time_delta']
            df['uin_in_time']=df['u_in_change']/df['time_delta']
            df.loc[df["time_step"] == 0, ["uin_in_time"]] = 0
            return df
            
        cols = ['u_in_change', 'area_u_in_abs', 'uin_in_time']
        self.train = create(train)[cols]
        self.test = create(test)[cols]
        
train, test = get_train_test()
U_in_delta().run().save()

In [23]:
class U_in_diff_maxmean(Feature):
    """
    https://www.kaggle.com/dlaststark/gb-vpp-whoppity-dub-dub
    """
    def create_features(self):
        def create(df):
            df['breath_id__u_in__diffmax'] = df.groupby(['breath_id'])['u_in'].transform('max') - df['u_in']
            df['breath_id__u_in__diffmean'] = df.groupby(['breath_id'])['u_in'].transform('mean') - df['u_in']
            return df
            
        cols = ['breath_id__u_in__diffmax', 'breath_id__u_in__diffmean']
        self.train = create(train)[cols]
        self.test = create(test)[cols]
        
train, test = get_train_test()
U_in_diff_maxmean().run().save()

[U_in_diff_maxmean] start
[U_in_diff_maxmean] done in 1 s


In [26]:
feats = ['base','U_in_delta','U_in_Lag_Diff']
train, test = load_datasets(feats)
train.head(50)

,id,breath_id,time_step,u_in,u_out,pressure,fold,u_in_change,area_u_in_abs,uin_in_time,u_in_lag1_diff,u_in_lag2_diff,u_in_lag3_diff,u_in_lag4_diff
0,1,1,0.000000,0.083334,0,5.837492,4,18.299707,0.000000,0.000000,0.083334,0.083334,0.083334,0.083334
1,2,1,0.033652,18.383041,0,5.907794,4,4.126236,0.138857,122.613777,18.299707,18.383041,18.383041,18.383041
2,3,1,0.067514,22.509278,0,7.876254,4,0.299544,0.010143,8.846007,4.126236,22.425944,22.509278,22.509278
3,4,1,0.101542,22.808822,0,11.742872,4,2.547028,0.086670,74.851355,0.299544,4.425781,22.725488,22.808822
4,5,1,0.135756,25.355850,0,12.234987,4,1.904016,0.065143,55.651320,2.547028,2.846573,6.972809,25.272516
5,6,1,0.169698,27.259866,0,12.867706,4,-0.132380,-0.004493,-3.900163,1.904016,4.451044,4.750588,8.876824
6,7,1,0.203708,27.127486,0,14.695562,4,-0.319754,-0.010875,-9.401713,-0.132380,1.771635,4.318663,4.618208
7,8,1,0.237723,26.807732,0,15.890699,4,1.056984,0.035953,31.074317,-0.319754,-0.452134,1.451881,3.998910
8,9,1,0.271776,27.864715,0,15.539188,4,0.448320,0.015267,13.165061,1.056984,0.737230,0.604850,2.508865
9,10,1,0.305732,28.313036,0,15.750094,4,-1.446278,-0.049109,-42.593529,0.448320,1.505304,1.185550,1.053170


In [22]:
#train, test = get_train_test()
#train['u_in_diff'] = train['u_in'].diff()
train['u_in_lag1'] = train.groupby('breath_id')['u_in'].shift(1)
train['u_in_diff1'] = train['u_in'] - train['u_in_lag1']
train

,id,breath_id,R,C,time_step,u_in,u_out,pressure,u_in_diff,u_in_lag1,u_in_diff1
0,1,1,20,50,0.000000,0.083334,0,5.837492,NaN,NaN,NaN
1,2,1,20,50,0.033652,18.383041,0,5.907794,18.299707,0.083334,18.299707
2,3,1,20,50,0.067514,22.509278,0,7.876254,4.126236,18.383041,4.126236
3,4,1,20,50,0.101542,22.808822,0,11.742872,0.299544,22.509278,0.299544
4,5,1,20,50,0.135756,25.355850,0,12.234987,2.547028,22.808822,2.547028
...,...,...,...,...,...,...,...,...,...,...,...
6035995,6035996,125749,50,10,2.504603,1.489714,1,3.869032,0.069002,1.420711,0.069002
6035996,6035997,125749,50,10,2.537961,1.488497,1,3.869032,-0.001217,1.489714,-0.001217
6035997,6035998,125749,50,10,2.571408,1.558978,1,3.798729,0.070481,1.488497,0.070481
6035998,6035999,125749,50,10,2.604744,1.272663,1,4.079938,-0.286315,1.558978,-0.286315


## Notebookサーベイしながら特徴量探し

In [ ]:
#https://www.kaggle.com/dlaststark/gb-vpp-whoppity-dub-dub
# よくわからん
df['ewm_u_in_mean'] = (df\
                       .groupby('breath_id')['u_in']\
                       .ewm(halflife=9)\
                       .mean()\
                       .reset_index(level=0,drop=True))
df[["15_in_sum","15_in_min","15_in_max","15_in_mean"]] = (df\
                                                          .groupby('breath_id')['u_in']\
                                                          .rolling(window=15,min_periods=1)\
                                                          .agg({"15_in_sum":"sum",
                                                                "15_in_min":"min",
                                                                "15_in_max":"max",
                                                                "15_in_mean":"mean"})\
                                                           .reset_index(level=0,drop=True))